In [1]:

from pathlib import Path
import shutil
import zipfile

# Kaggle の入力データセット slug（username/slug の slug 部分）
DATASET_SLUG = 'gns-codes'
DATASET_ROOT = Path(f'/kaggle/input/{DATASET_SLUG}')
WORK_ROOT = Path('/kaggle/working')
repo_dir = WORK_ROOT / 'code'

code_dir = DATASET_ROOT / 'code'
code_zip = DATASET_ROOT / 'code.zip'

# 展開済み code/ を優先し、無ければ code.zip を展開
if code_dir.exists():
    src = code_dir
elif code_zip.exists():
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    with zipfile.ZipFile(code_zip) as zf:
        zf.extractall(repo_dir)
    src = repo_dir
else:
    raise FileNotFoundError(f"/kaggle/input/{DATASET_SLUG} に code も code.zip もありません。Add Data で {DATASET_SLUG} を追加してください。")

# フラット化（zip解凍で1階層挟まった場合）
if src != repo_dir:
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    shutil.copytree(src, repo_dir)
if repo_dir.exists():
    children = list(repo_dir.iterdir())
    if len(children) == 1 and children[0].is_dir():
        inner = children[0]
        for p in inner.iterdir():
            p.rename(repo_dir / p.name)
        inner.rmdir()

%cd $repo_dir


FileNotFoundError: /kaggle/input/gns-codes に code も code.zip もありません。Add Data で gns-codes を追加してください。

In [ ]:

# PyG の radius_graph で必要になる torch-cluster だけインストール
import torch

torch_ver = torch.__version__.split('+')[0]
cuda_ver = torch.version.cuda
cuda_tag = 'cpu' if cuda_ver is None else 'cu' + cuda_ver.replace('.', '')
url = f"https://data.pyg.org/whl/torch-{torch_ver}+{cuda_tag}.html"
print('PyG wheels:', url)

!pip -q install torch-cluster -f {url}
!pip -q install torch_geometric


In [ ]:
# 日本語表示用パッケージのインストール
!pip -q install japanize-matplotlib

In [ ]:
from pathlib import Path
import copy
import yaml
import re
import json

REPO = Path('/kaggle/working/code')
cfg_path = REPO / 'config_rollout.yaml'

# 全モデルを同じデータセットで検証したい場合のスイッチ
USE_FORCE_DATASET = True  # True にすると下記 FORCE_DATASET_ROOT を強制使用
FORCE_DATASET_ROOT = Path('/kaggle/input/dam-break-left-800')

# モデルごとに対応するデータセットを自動解決する

def _load_metadata(path: Path):
    try:
        with path.open('r', encoding='utf-8') as f:
            return json.load(f)
    except Exception:
        return None

def _find_metadata(root: Path):
    candidates = [
        root / 'metadata.json',
        root / 'metadata' / 'metadata.json',
    ]
    candidates += list(root.glob('**/metadata*.json'))
    for c in candidates:
        if c.is_file():
            meta = _load_metadata(c)
            if isinstance(meta, dict):
                return meta, c
    return None, None

def _extract_sequence_length(meta: dict):
    if not isinstance(meta, dict):
        return None
    for key in ('train', 'valid', 'test', 'rollout'):
        part = meta.get(key)
        if isinstance(part, dict) and 'sequence_length' in part:
            try:
                return int(part['sequence_length'])
            except Exception:
                pass
    if 'sequence_length' in meta:
        try:
            return int(meta['sequence_length'])
        except Exception:
            return None
    return None

def resolve_dataset_root(step: int | None):
    base = Path('/kaggle/input')
    if not base.exists():
        return None, False

    if USE_FORCE_DATASET:
        if FORCE_DATASET_ROOT.exists():
            print(f"[force] step={step} を無視して {FORCE_DATASET_ROOT} を使用します。")
            return FORCE_DATASET_ROOT, True
        else:
            print(f"[force] {FORCE_DATASET_ROOT} が見つかりません。通常の解決ロジックに戻ります。")

    # 1) ディレクトリ名にステップ数が含まれるものを優先
    if step is not None:
        for dataset_root in sorted(base.iterdir()):
            name = dataset_root.name
            if (name.endswith(f'-{step}') or name.endswith(f'_{step}')
                    or f'-{step}-' in name or f'_{step}_' in name):
                return dataset_root, False

    # 2) metadata の sequence_length で一致を探す
    for dataset_root in sorted(base.iterdir()):
        meta, meta_path = _find_metadata(dataset_root)
        if not meta:
            continue
        seq = _extract_sequence_length(meta)
        if step is not None and seq == step:
            return dataset_root, False

    # 3) フォールバック（もっとも汎用な800データセット）
    fallback = base / 'dam-break-left-800'
    if fallback.exists():
        print(f"[fallback] step={step} に一致するデータセットが見つからず、{fallback} を使用します。")
        return fallback, True
    print(f"[fallback] step={step} に一致するデータセットも既定の dam-break-left-800 も見つかりません。")
    return None, True

# 出力設定（共通）
output_root = REPO / 'rollouts'
viz_format = 'html'  # html|mp4|gif

with cfg_path.open('r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

# データセットはモデルごとに後続セルで差し替える
cfg['method'] = 'gns'
cfg['rollout_inference_max_examples'] = None    # 検証データ全件で推論し平均を取る
cfg['output_path'] = str(output_root)
cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = '/kaggle/input'  # placeholder


def _get_nested(d, keys):
    cur = d
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return None
        cur = cur[k]
    return cur

_raw_dt_candidates = [
    _get_nested(cfg, ['scenario_options', 'fluid', 'integrator', 'dt']),
    _get_nested(cfg, ['integrator', 'dt']),
    _get_nested(cfg, ['method_options', 'hamiltonian_sph', 'integrator', 'dt']),
]
_raw_dt = next((v for v in _raw_dt_candidates if v is not None), 0.006)
try:
    INFER_DT = float(_raw_dt)
    if INFER_DT <= 0:
        raise ValueError
except Exception:
    INFER_DT = 0.006

# モデルは後続セルで差し替える（ベースを保存しておく）
cfg['model_path'] = str(REPO / 'models')
cfg['model_file'] = None
cfg['output_filename'] = 'rollout'

BASE_CFG = copy.deepcopy(cfg)

with cfg_path.open('w', encoding='utf-8') as f:
    yaml.safe_dump(cfg, f, allow_unicode=True)

print('ベース設定を書き出しました:', cfg_path)
print('output_path:', cfg['output_path'])
print('inference dt:', INFER_DT)


In [ ]:
%cd /kaggle/working/code

import copy
import re
import subprocess
import json
from pathlib import Path
import numpy as np
import torch
import sys
sys.path.append(str(Path('/kaggle/working/code/src')))

from analyze_rollouts import analyze_rollout
from simulator_factory import _get_simulator
from train_config import load_config, INPUT_SEQUENCE_LENGTH, KINEMATIC_PARTICLE_ID
from train_paths import _resolve_model_path
from train_utils import _resolve_rollout_dataset_path
import data_loader
import reading_utils

# 比較対象モデルを集約（rollout_diff 優先）
model_roots = [
    REPO / 'models' / 'dataset_diff',
    REPO / 'models',
]
model_files: list[Path] = []
for root in model_roots:
    if not root.exists():
        continue
    for path in root.glob('*.pt'):
        model_files.append(path.resolve())

# 重複除去 & ソート（親ディレクトリ優先→ステップ番号）
seen = set()
unique_models: list[Path] = []
for p in model_files:
    if p in seen:
        continue
    seen.add(p)
    unique_models.append(p)

def _step_key(p: Path):
    m = re.search(r'(\d+)', p.stem)
    step = int(m.group(1)) if m else -1
    priority = 0 if p.parent.name == "dataset_diff" else 1
    return (priority, p.parent.name, step)

def _step_from_path(p: Path):
    m = re.search(r'(\d+)(?!.*\d)', p.stem)
    return int(m.group(1)) if m else None

model_files = sorted(unique_models, key=_step_key)

if not model_files:
    raise FileNotFoundError('*.pt が見つかりません。models/ 下にモデルを配置してください。')

print('評価対象モデル:')
for p in model_files:
    print(' -', p)


# キャッシュ読み込み/保存用
cache_path = output_root / 'inference_results.json'
results = []

if cache_path.exists():
    with cache_path.open('r', encoding='utf-8') as f:
        saved = json.load(f)
    results = saved.get('results', [])
    print(f"キャッシュを読み込みました: {cache_path}")
    print(f"キャッシュ内のモデル数: {len(results)}")
else:
    def compute_one_step_error(cfg_path: Path, device: torch.device, dt: float):
        # 検証データ全軌跡で1-step誤差を平均
        cfg_obj = load_config(str(cfg_path))
        dataset_path = _resolve_rollout_dataset_path(cfg_obj)
        if dataset_path is None:
            raise FileNotFoundError(
                f"rollout/valid/test/train の npz が見つかりません: {cfg_obj.data_path}"
            )

        metadata_key = (
            cfg_obj.active_scenario.rollout_metadata_split
            if getattr(cfg_obj, 'active_scenario', None)
            and cfg_obj.active_scenario.rollout_metadata_split
            else 'rollout'
        )
        metadata = reading_utils.read_metadata(cfg_obj.data_path, metadata_key)
        if isinstance(metadata, dict) and 'acc_mean' not in metadata:
            metadata = metadata.get('rollout') or metadata.get('train') or metadata
        simulator = _get_simulator(
            metadata,
            cfg_obj.noise_std,
            cfg_obj.noise_std,
            device,
            cfg_obj,
        )
        model_file = _resolve_model_path(cfg_obj)
        simulator.load(model_file)
        simulator.to(device)
        simulator.eval()

        loader = data_loader.get_data_loader_by_trajectories(dataset_path)

        step_sums = None
        vel_sums = None
        acc_sums = None
        step_counts = None
        n_rollouts = 0

        dt = float(dt) if dt and dt > 0 else 1.0

        with torch.no_grad():
            for traj in loader:
                if len(traj) == 4:
                    positions, particle_type, material_property, n_particles = traj
                    material_property = material_property.to(device)
                else:
                    positions, particle_type, n_particles = traj
                    material_property = None

                positions = positions.to(device)
                particle_type = particle_type.to(device)
                n_particles_tensor = torch.tensor(
                    [int(n_particles)], device=device, dtype=torch.int32
                )

                total_steps_in_traj = positions.shape[1] - INPUT_SEQUENCE_LENGTH
                if total_steps_in_traj <= 0:
                    continue

                step_errors: list[float] = []
                vel_errors: list[float] = []
                acc_errors: list[float] = []
                for t in range(total_steps_in_traj):
                    window = positions[:, t : t + INPUT_SEQUENCE_LENGTH]
                    target = positions[:, t + INPUT_SEQUENCE_LENGTH]

                    pred = simulator.predict_positions(
                        window,
                        nparticles_per_example=n_particles_tensor,
                        particle_types=particle_type,
                        material_property=material_property,
                    )

                    # 運動制約粒子は GT を強制
                    kinematic_mask = (particle_type == KINEMATIC_PARTICLE_ID).to(device)
                    pred = torch.where(kinematic_mask[:, None], target, pred)

                    dist = torch.sqrt(((pred - target) ** 2).sum(dim=-1))
                    vel_pred = (pred - window[:, -1]) / dt
                    vel_target = (target - window[:, -1]) / dt
                    vel_dist = torch.sqrt(((vel_pred - vel_target) ** 2).sum(dim=-1))

                    # 加速度は2階差分
                    accel_pred = (pred - 2 * window[:, -1] + window[:, -2]) / (dt**2)
                    accel_target = (target - 2 * window[:, -1] + window[:, -2]) / (dt**2)
                    accel_dist = torch.sqrt(((accel_pred - accel_target) ** 2).sum(dim=-1))

                    step_errors.append(float(dist.mean().item()))
                    vel_errors.append(float(vel_dist.mean().item()))
                    acc_errors.append(float(accel_dist.mean().item()))

                if step_errors:
                    n_rollouts += 1
                    if step_sums is None:
                        step_sums = np.zeros(len(step_errors), dtype=float)
                        vel_sums = np.zeros(len(step_errors), dtype=float)
                        acc_sums = np.zeros(len(step_errors), dtype=float)
                        step_counts = np.zeros(len(step_errors), dtype=int)
                    elif len(step_errors) > len(step_sums):
                        # 可変長シーケンスに対応してパディング
                        pad_len = len(step_errors) - len(step_sums)
                        step_sums = np.pad(step_sums, (0, pad_len))
                        vel_sums = np.pad(vel_sums, (0, pad_len))
                        acc_sums = np.pad(acc_sums, (0, pad_len))
                        step_counts = np.pad(step_counts, (0, pad_len))

                    step_sums[: len(step_errors)] += step_errors
                    vel_sums[: len(step_errors)] += vel_errors
                    acc_sums[: len(step_errors)] += acc_errors
                    step_counts[: len(step_errors)] += 1

        if not n_rollouts or step_sums is None or vel_sums is None or acc_sums is None:
            return None

        valid_mask = step_counts > 0
        mean_per_timestep = (step_sums[valid_mask] / step_counts[valid_mask]).tolist()
        mean_vel_per_timestep = (vel_sums[valid_mask] / step_counts[valid_mask]).tolist()
        mean_acc_per_timestep = (acc_sums[valid_mask] / step_counts[valid_mask]).tolist()
        mean_distance_error = float(step_sums.sum() / step_counts.sum())
        mean_velocity_error = float(vel_sums.sum() / step_counts.sum())
        mean_acceleration_error = float(acc_sums.sum() / step_counts.sum())
        used_length = int(valid_mask.sum())

        return {
            'per_timestep': mean_per_timestep,
            'velocity_per_timestep': mean_vel_per_timestep,
            'acceleration_per_timestep': mean_acc_per_timestep,
            'mean_distance_error': mean_distance_error,
            'mean_velocity_error': mean_velocity_error,
            'mean_acceleration_error': mean_acceleration_error,
            'n_rollouts': n_rollouts,
            'used_length': used_length,
        }


    cmd = ['python', 'src/train.py', '--config', str(cfg_path)]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for model_path in model_files:
        tag = model_path.stem  # ディレクトリ名を含めずモデル名のみ
        step_num = _step_from_path(model_path)
        dataset_root, used_fallback = resolve_dataset_root(step_num)
        if dataset_root is None:
            raise FileNotFoundError(f"対応するデータセットが見つかりません（step={step_num}）")

        cfg = copy.deepcopy(BASE_CFG)
        cfg['model_path'] = str(model_path.parent)
        cfg['model_file'] = str(model_path)
        cfg['output_filename'] = f"rollout_{tag}"
        cfg.setdefault('scenario_options', {}).setdefault('fluid', {})['dataset'] = str(dataset_root)
        cfg['data_path'] = str(dataset_root)
        cfg['rollout_dataset'] = 'valid'

        with cfg_path.open('w', encoding='utf-8') as f:
            yaml.safe_dump(cfg, f, allow_unicode=True)

        print(f"=== {tag} ===")
        print('dataset:', dataset_root)
        if used_fallback:
            print('[warn] 指定ステップに一致するデータセットが見つからなかったためフォールバックを使用しました。')
        print('Running:', ' '.join(cmd))
        subprocess.run(cmd, check=True)

        output_dir = Path(cfg['output_path']) / cfg['method'] / cfg['output_filename']
        pkl_files = sorted(output_dir.glob(f"{cfg['output_filename']}_ex*.pkl"))
        if not pkl_files:
            raise FileNotFoundError(f"{output_dir} に *_ex*.pkl がありません。推論が成功したか確認してください。")

        rollout_dists = []
        rollout_vels = []
        rollout_accs = []
        for pkl_file in pkl_files:
            res_one = analyze_rollout(pkl_file, dt=INFER_DT)
            dist = np.array(res_one['distance_error_per_timestep'])
            rollout_dists.append(dist)
            vel_raw = res_one.get('velocity_error_per_timestep')
            vel = np.array(vel_raw) if vel_raw is not None else np.array([])
            if vel.size:
                rollout_vels.append(vel)
            acc_raw = res_one.get('acceleration_error_per_timestep')
            acc = np.array(acc_raw) if acc_raw is not None else np.array([])
            if acc.size:
                rollout_accs.append(acc)

        max_len = max(len(d) for d in rollout_dists)
        dist_sums = np.zeros(max_len)
        dist_counts = np.zeros(max_len, dtype=int)
        for dist in rollout_dists:
            dist_sums[: len(dist)] += dist
            dist_counts[: len(dist)] += 1

        valid_mask = dist_counts > 0
        mean_dist_per_timestep = (dist_sums[valid_mask] / dist_counts[valid_mask]).tolist()
        weighted_mean = float(dist_sums.sum() / dist_counts.sum())
        used_length = int(valid_mask.sum())
        avg_steps = float(sum(len(d) for d in rollout_dists) / len(rollout_dists))

        if rollout_vels:
            max_len_vel = max(len(v) for v in rollout_vels)
            vel_sums = np.zeros(max_len_vel)
            vel_counts = np.zeros(max_len_vel, dtype=int)
            for vel in rollout_vels:
                vel_sums[: len(vel)] += vel
                vel_counts[: len(vel)] += 1
            valid_mask_vel = vel_counts > 0
            mean_vel_per_timestep = (vel_sums[valid_mask_vel] / vel_counts[valid_mask_vel]).tolist()
            weighted_mean_vel = float(vel_sums.sum() / vel_counts.sum())
            used_length_vel = int(valid_mask_vel.sum())
        else:
            mean_vel_per_timestep = []
            weighted_mean_vel = None
            used_length_vel = 0

        if rollout_accs:
            max_len_acc = max(len(a) for a in rollout_accs)
            acc_sums = np.zeros(max_len_acc)
            acc_counts = np.zeros(max_len_acc, dtype=int)
            for acc in rollout_accs:
                acc_sums[: len(acc)] += acc
                acc_counts[: len(acc)] += 1
            valid_mask_acc = acc_counts > 0
            mean_acc_per_timestep = (acc_sums[valid_mask_acc] / acc_counts[valid_mask_acc]).tolist()
            weighted_mean_acc = float(acc_sums.sum() / acc_counts.sum())
            used_length_acc = int(valid_mask_acc.sum())
        else:
            mean_acc_per_timestep = []
            weighted_mean_acc = None
            used_length_acc = 0

        one_step_res = compute_one_step_error(cfg_path, device, dt=INFER_DT)
        if one_step_res:
            print(
                f"  one-step: rollouts={one_step_res['n_rollouts']} / mean distance error={one_step_res['mean_distance_error']:.6f} / mean velocity error={one_step_res['mean_velocity_error']:.6f} / mean acc error={one_step_res['mean_acceleration_error']:.6f}"
            )
        else:
            print('  one-step: 計算できませんでした。')

        vel_msg = f"{weighted_mean_vel:.6f}" if weighted_mean_vel is not None else 'N/A'
        acc_msg = f"{weighted_mean_acc:.6f}" if weighted_mean_acc is not None else 'N/A'
        results.append({
            'tag': tag,
            'distance_error_per_timestep': mean_dist_per_timestep,
            'mean_distance_error': weighted_mean,
            'n_rollouts': len(rollout_dists),
            'used_length': used_length,
            'pkl_dir': str(output_dir),
            'dataset_root': str(dataset_root),
            'step_num': step_num,
            'used_fallback': used_fallback,
            'velocity_error_per_timestep': mean_vel_per_timestep,
            'mean_velocity_error': weighted_mean_vel,
            'velocity_used_length': used_length_vel,
            'acceleration_error_per_timestep': mean_acc_per_timestep,
            'mean_acceleration_error': weighted_mean_acc,
            'acceleration_used_length': used_length_acc,
            'one_step_error_per_timestep': one_step_res['per_timestep'] if one_step_res else [],
            'one_step_mean_distance_error': one_step_res['mean_distance_error'] if one_step_res else None,
            'one_step_velocity_error_per_timestep': one_step_res['velocity_per_timestep'] if one_step_res else [],
            'one_step_mean_velocity_error': one_step_res['mean_velocity_error'] if one_step_res else None,
            'one_step_acceleration_error_per_timestep': one_step_res['acceleration_per_timestep'] if one_step_res else [],
            'one_step_mean_acceleration_error': one_step_res['mean_acceleration_error'] if one_step_res else None,
            'one_step_used_length': one_step_res['used_length'] if one_step_res else 0,
            'one_step_n_rollouts': one_step_res['n_rollouts'] if one_step_res else 0,
        })

        print(
            f"  rollout: rollouts={len(rollout_dists)} / avg steps per rollout={avg_steps:.1f} / mean distance error={weighted_mean:.6f} / mean velocity error={vel_msg} / mean acc error={acc_msg} (valid 平均)"
        )

    # 推論結果をキャッシュ保存
    cache_path.parent.mkdir(parents=True, exist_ok=True)
    with cache_path.open('w', encoding='utf-8') as f:
        json.dump({'results': results, 'infer_dt': INFER_DT}, f, ensure_ascii=False, indent=2)
    print('結果をキャッシュに保存しました:', cache_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import cycle

# 日本語表示設定
try:
    import japanize_matplotlib
except ImportError:
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

MAX_SUMMARY_STEPS = 100
# None なら上限なし
MAX_PLOT_STEPS = None  # 1ステップ誤差の表示上限

# ラベル・凡例のサイズ設定（大きめ）
LABEL_FONTSIZE = 16
TICK_FONTSIZE = 14
LEGEND_FONTSIZE = 16
TITLE_FONTSIZE = 18

if not results:
    print('results が空です。前のセルを実行してください。')
else:
    # モデルごとに固定の色を割り当て（rollout と 1step で揃える）
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key().get('color', [])
    if not color_cycle:
        color_cycle = list(plt.cm.tab10.colors)
    color_iter = cycle(color_cycle)
    color_map = {r['tag']: next(color_iter) for r in results}

    # -------- 全タイムステップの距離誤差（rollout 平均） --------
    plt.figure(figsize=(10, 6))
    max_len_rollout = max(len(np.array(r['distance_error_per_timestep'])) for r in results)
    for r in results:
        err = np.array(r['distance_error_per_timestep'])
        timesteps = np.arange(len(err))
        # 短い系列ほど zorder を高くして上に重ねる
        z = max_len_rollout - len(err)
        plt.plot(
            timesteps,
            err,
            label=f"{r['tag']} (L={len(err)})",
            color=color_map[r['tag']],
            zorder=z,
        )
    plt.xlabel('タイムステップ', fontsize=LABEL_FONTSIZE)
    plt.ylabel("ロールアウト誤差(20サンプル平均)", fontsize=LABEL_FONTSIZE)
    plt.title('タイムステップ数を変えたときのロールアウト誤差）', fontsize=TITLE_FONTSIZE)
    plt.grid(True, alpha=0.3)
    plt.tick_params(labelsize=TICK_FONTSIZE)
    plt.legend(fontsize=LEGEND_FONTSIZE)
    plt.tight_layout()

    plot_path = output_root / 'distance_error_comparison.png'
    plt.savefig(plot_path, dpi=150)
    plt.show()
    print('プロットを保存:', plot_path)

    # -------- 全タイムステップの速度誤差（rollout 平均） --------
    vel_plot_items = []
    for r in results:
        err = np.array(r.get('velocity_error_per_timestep') or [])
        if err.size == 0:
            continue
        vel_plot_items.append((len(err), err, r))
    if vel_plot_items:
        plt.figure(figsize=(10, 6))
        max_len_rollout_vel = max(item[0] for item in vel_plot_items)
        for _, err, r in vel_plot_items:
            timesteps = np.arange(len(err))
            z = max_len_rollout_vel - len(err)
            plt.plot(
                timesteps,
                err,
                label=f"{r['tag']} (L={len(err)})",
                color=color_map[r['tag']],
                zorder=z,
            )
        plt.xlabel('タイムステップ', fontsize=LABEL_FONTSIZE)
        plt.ylabel('1ステップ誤差', fontsize=LABEL_FONTSIZE)
        plt.title("タイムステップ数を変えたときの1ステップ誤差", fontsize=TITLE_FONTSIZE)
        plt.grid(True, alpha=0.3)
        plt.tick_params(labelsize=TICK_FONTSIZE)
        plt.legend(fontsize=LEGEND_FONTSIZE)
        plt.tight_layout()

        plot_path_vel = output_root / 'velocity_error_comparison.png'
        plt.savefig(plot_path_vel, dpi=150)
        plt.show()
        print('速度誤差プロットを保存:', plot_path_vel)

    # -------- 全タイムステップの加速度誤差（rollout 平均） --------
    acc_plot_items = []
    for r in results:
        err = np.array(r.get('acceleration_error_per_timestep') or [])
        if err.size == 0:
            continue
        acc_plot_items.append((len(err), err, r))
    if acc_plot_items:
        plt.figure(figsize=(10, 6))
        max_len_rollout_acc = max(item[0] for item in acc_plot_items)
        for _, err, r in acc_plot_items:
            timesteps = np.arange(len(err))
            z = max_len_rollout_acc - len(err)
            plt.plot(
                timesteps,
                err,
                label=f"{r['tag']} (L={len(err)}, ロールアウト本数={r['n_rollouts']})",
                color=color_map[r['tag']],
                zorder=z,
            )
        plt.xlabel('タイムステップ', fontsize=LABEL_FONTSIZE)
        plt.ylabel('加速度誤差の平均（検証データ平均）', fontsize=LABEL_FONTSIZE)
        plt.title('ロールアウト加速度誤差（タイムステップ別、検証平均）', fontsize=TITLE_FONTSIZE)
        plt.grid(True, alpha=0.3)
        plt.tick_params(labelsize=TICK_FONTSIZE)
        plt.legend(fontsize=LEGEND_FONTSIZE)
        plt.tight_layout()

        plot_path_acc = output_root / 'acceleration_error_comparison.png'
        plt.savefig(plot_path_acc, dpi=150)
        plt.show()
        print('加速度誤差プロットを保存:', plot_path_acc)

    # -------- 1ステップ誤差（教師あり1-step, validation 平均） --------
    plot_items = []
    for r in results:
        err = np.array(r.get('one_step_error_per_timestep') or [])
        if err.size == 0:
            continue
        if MAX_PLOT_STEPS is not None:
            err = err[:MAX_PLOT_STEPS]
        # results の順序を維持して rollouts と同じ並びにする
        plot_items.append((len(err), err, r))

    if plot_items:
        plt.figure(figsize=(10, 6))
        max_len_onestep = max(len(item[1]) for item in plot_items)
        for _, err, r in plot_items:
            timesteps = np.arange(len(err))
            z = max_len_onestep - len(err)
            plt.plot(
                timesteps,
                err,
                marker='o',
                markersize=3,
                linewidth=1.2,
                label=f"{r['tag']} (L={len(err)}, 1ステップ本数={r['one_step_n_rollouts']})",
                color=color_map[r['tag']],
                zorder=z,
            )
        plt.xlabel('タイムステップ', fontsize=LABEL_FONTSIZE)
        plt.ylabel('距離誤差の平均（1ステップ、検証平均）', fontsize=LABEL_FONTSIZE)
        plt.title('1ステップ距離誤差（タイムステップ別、検証平均）', fontsize=TITLE_FONTSIZE)
        plt.grid(True, alpha=0.3)
        plt.tick_params(labelsize=TICK_FONTSIZE)
        plt.legend(fontsize=LEGEND_FONTSIZE)
        plt.tight_layout()

        plot_path_step = output_root / 'distance_error_per_timestep.png'
        plt.savefig(plot_path_step, dpi=150)
        plt.show()
        print('1ステップ距離誤差プロットを保存:', plot_path_step)

    # -------- 1ステップの速度誤差（validation 平均） --------
    vel_step_items = []
    for r in results:
        err = np.array(r.get('one_step_velocity_error_per_timestep') or [])
        if err.size == 0:
            continue
        if MAX_PLOT_STEPS is not None:
            err = err[:MAX_PLOT_STEPS]
        vel_step_items.append((len(err), err, r))

    if vel_step_items:
        plt.figure(figsize=(10, 6))
        max_len_onestep_vel = max(len(item[1]) for item in vel_step_items)
        for _, err, r in vel_step_items:
            timesteps = np.arange(len(err))
            z = max_len_onestep_vel - len(err)
            plt.plot(
                timesteps,
                err,
                marker='o',
                markersize=3,
                linewidth=1.2,
                label=f"{r['tag']} (L={len(err)}, 1ステップ本数={r['one_step_n_rollouts']})",
                color=color_map[r['tag']],
                zorder=z,
            )
        plt.xlabel('タイムステップ', fontsize=LABEL_FONTSIZE)
        plt.ylabel('速度誤差の平均（1ステップ、検証平均）', fontsize=LABEL_FONTSIZE)
        plt.title('1ステップ速度誤差（タイムステップ別、検証平均）', fontsize=TITLE_FONTSIZE)
        plt.grid(True, alpha=0.3)
        plt.tick_params(labelsize=TICK_FONTSIZE)
        plt.legend(fontsize=LEGEND_FONTSIZE)
        plt.tight_layout()

        plot_path_vel_step = output_root / 'velocity_error_per_timestep.png'
        plt.savefig(plot_path_vel_step, dpi=150)
        plt.show()
        print('1ステップ速度誤差プロットを保存:', plot_path_vel_step)

    # -------- 1ステップの加速度誤差（validation 平均） --------
    acc_step_items = []
    for r in results:
        err = np.array(r.get('one_step_acceleration_error_per_timestep') or [])
        if err.size == 0:
            continue
        if MAX_PLOT_STEPS is not None:
            err = err[:MAX_PLOT_STEPS]
        acc_step_items.append((len(err), err, r))

    if acc_step_items:
        plt.figure(figsize=(10, 6))
        max_len_onestep_acc = max(len(item[1]) for item in acc_step_items)
        for _, err, r in acc_step_items:
            timesteps = np.arange(len(err))
            z = max_len_onestep_acc - len(err)
            plt.plot(
                timesteps,
                err,
                marker='o',
                markersize=3,
                linewidth=1.2,
                label=f"{r['tag']} (L={len(err)}, 1ステップ本数={r['one_step_n_rollouts']})",
                color=color_map[r['tag']],
                zorder=z,
            )
        plt.xlabel('タイムステップ', fontsize=LABEL_FONTSIZE)
        plt.ylabel('加速度誤差の平均（1ステップ、検証平均）', fontsize=LABEL_FONTSIZE)
        plt.title('1ステップ加速度誤差（タイムステップ別、検証平均）', fontsize=TITLE_FONTSIZE)
        plt.grid(True, alpha=0.3)
        plt.tick_params(labelsize=TICK_FONTSIZE)
        plt.legend(fontsize=LEGEND_FONTSIZE)
        plt.tight_layout()

        plot_path_acc_step = output_root / 'acceleration_error_per_timestep.png'
        plt.savefig(plot_path_acc_step, dpi=150)
        plt.show()
        print('1ステップ加速度誤差プロットを保存:', plot_path_acc_step)

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# 日本語表示設定
try:
    import japanize_matplotlib
except ImportError:
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

DT = INFER_DT  # 推論と同じ刻み幅を使用
target_name = 'dam-break-left-800'

if not results:
    print('results が空です。前のセルを実行してください。')
else:
    target = [r for r in results if target_name in str(r.get('dataset_root', ''))]
    if not target:
        print(f"{target_name} を含むデータセットの結果が見つかりません。先に FORCE を有効化するか、対象モデルだけ実行してください。")
    else:
        color_cycle = plt.rcParams['axes.prop_cycle'].by_key().get('color', []) or list(plt.cm.tab10.colors)
        color_map = {r['tag']: color_cycle[i % len(color_cycle)] for i, r in enumerate(target)}

        def _load_rms(pkl_path: Path):
            with pkl_path.open('rb') as f:
                data = pickle.load(f)
            pred = data['predicted_rollout']
            gt = data['ground_truth_rollout']
            if pred.ndim == 4:
                pred = pred[:, 0]
            if gt.ndim == 4:
                gt = gt[:, 0]
            v_pred = np.diff(pred, axis=0) / DT
            v_gt = np.diff(gt, axis=0) / DT
            a_pred = np.diff(v_pred, axis=0) / DT
            a_gt = np.diff(v_gt, axis=0) / DT
            v_rms_pred = np.sqrt((v_pred ** 2).mean(axis=(1, 2)))
            v_rms_gt = np.sqrt((v_gt ** 2).mean(axis=(1, 2)))
            a_rms_pred = np.sqrt((a_pred ** 2).mean(axis=(1, 2)))
            a_rms_gt = np.sqrt((a_gt ** 2).mean(axis=(1, 2)))
            return v_rms_pred, v_rms_gt, a_rms_pred, a_rms_gt

        fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
        for r in target:
            pdir = Path(r['pkl_dir'])
            pkls = sorted(pdir.glob('*_ex*.pkl'))
            if not pkls:
                print('pkl が見つかりません:', pdir)
                continue
            v_pred, v_gt, a_pred, a_gt = _load_rms(pkls[0])
            t_v = np.arange(len(v_pred))
            t_a = np.arange(len(a_pred))
            color = color_map[r['tag']]
            axes[0].plot(t_v, v_gt, linestyle='--', color=color, alpha=0.8, label=f"{r['tag']} 正解")
            axes[0].plot(t_v, v_pred, linestyle='-', color=color, label=f"{r['tag']} 予測")
            axes[1].plot(t_a, a_gt, linestyle='--', color=color, alpha=0.8, label=f"{r['tag']} 正解")
            axes[1].plot(t_a, a_pred, linestyle='-', color=color, label=f"{r['tag']} 予測")

        axes[0].set_ylabel('速度RMS')
        axes[0].set_title(f'速度RMSの時間変化)')
        axes[0].grid(True, alpha=0.3)
        axes[1].set_xlabel('タイムステップ')
        axes[1].set_ylabel('加速度RMS')
        axes[1].set_title(f'加速度RMSの時間変化')
        axes[1].grid(True, alpha=0.3)

        handles, labels = axes[0].get_legend_handles_labels()
        axes[0].legend(handles, labels, loc='upper right')
        handles, labels = axes[1].get_legend_handles_labels()
        axes[1].legend(handles, labels, loc='upper right')

        fig.tight_layout()
        out_path = output_root / f'rms_{target_name}.png'
        fig.savefig(out_path, dpi=150)
        plt.show()
        print('RMS プロットを保存:', out_path)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 日本語表示設定
try:
    import japanize_matplotlib
except ImportError:
    plt.rcParams['font.sans-serif'] = ['Noto Sans CJK JP', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

TARGET_STEP = 800
MA_WINDOW = 50  # 移動平均の窓幅（低周波成分用）

def moving_average(arr: np.ndarray, window: int) -> np.ndarray:
    if arr.size == 0 or window <= 1:
        return arr.astype(float)
    window = min(window, len(arr))
    kernel = np.ones(window, dtype=float) / window
    return np.convolve(arr, kernel, mode='same')

def split_low_high(arr: np.ndarray, window: int) -> tuple[np.ndarray, np.ndarray]:
    low = moving_average(arr, window)
    high = arr.astype(float) - low
    return low, high

def rms(x: np.ndarray) -> float:
    if x.size == 0:
        return float('nan')
    return float(np.sqrt(np.mean(x ** 2)))

if not results:
    print('results が空です。前のセルを実行してください。')
else:
    target_models = [r for r in results if r.get('step_num') == TARGET_STEP]
    if not target_models:
        print(f"{TARGET_STEP} ステップのモデルが見つかりません。step_num を確認してください。")
    else:
        color_cycle = plt.rcParams['axes.prop_cycle'].by_key().get('color', []) or list(plt.cm.tab10.colors)
        color_map = {r['tag']: color_cycle[i % len(color_cycle)] for i, r in enumerate(target_models)}

        fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=False)
        summary = []

        for r in target_models:
            tag = r['tag']
            vel = np.array(r.get('one_step_velocity_error_per_timestep') or [])
            acc = np.array(r.get('one_step_acceleration_error_per_timestep') or [])

            if vel.size:
                low_v, high_v = split_low_high(vel, MA_WINDOW)
                t_v = np.arange(len(vel))
                axes[0].plot(t_v, low_v, color=color_map[tag], label=f"{tag} 低周波")
                axes[0].plot(t_v, high_v, color=color_map[tag], linestyle='--', alpha=0.7, label=f"{tag} 高周波")
                summary.append(f"{tag} 速度: 低周波RMS={rms(low_v):.6f}, 高周波RMS={rms(high_v):.6f}")
            else:
                summary.append(f"{tag} 速度: データなし")

            if acc.size:
                low_a, high_a = split_low_high(acc, MA_WINDOW)
                t_a = np.arange(len(acc))
                axes[1].plot(t_a, low_a, color=color_map[tag], label=f"{tag} 低周波")
                axes[1].plot(t_a, high_a, color=color_map[tag], linestyle='--', alpha=0.7, label=f"{tag} 高周波")
                summary.append(f"{tag} 加速度: 低周波RMS={rms(low_a):.6f}, 高周波RMS={rms(high_a):.6f}")
            else:
                summary.append(f"{tag} 加速度: データなし")

        axes[0].set_title(f'1ステップ速度誤差の低/高周波分離 (step={TARGET_STEP}, 窓={MA_WINDOW})')
        axes[0].set_xlabel('タイムステップ')
        axes[0].set_ylabel('速度誤差')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend(fontsize=12)

        axes[1].set_title(f'1ステップ加速度誤差の低/高周波分離 (step={TARGET_STEP}, 窓={MA_WINDOW})')
        axes[1].set_xlabel('タイムステップ')
        axes[1].set_ylabel('加速度誤差')
        axes[1].grid(True, alpha=0.3)
        axes[1].legend(fontsize=12)

        fig.tight_layout()
        out_path = output_root / f'one_step_low_high_step{TARGET_STEP}.png'
        fig.savefig(out_path, dpi=150)
        plt.show()
        print('分離プロットを保存:', out_path)
        if summary:
            print('RMS サマリ:')
            for line in summary:
                print(' -', line)